This model Generate the Answer & Not stored the Data

In [8]:
import os
from openai import OpenAI
from docx import Document
from fuzzywuzzy import fuzz

OpenAI.api_key = os.environ["OPENAI_API_KEY"]
client = OpenAI()

def read_qa_from_docx(docx_path):
    doc = Document(docx_path)
    qa_dict = {}

    current_question = None
    for paragraph in doc.paragraphs:
        text = paragraph.text.strip()

        if text.endswith("?"):
            current_question = text
        elif current_question and text.endswith("."):
            qa_dict[current_question] = text
            current_question = None

    return qa_dict

def find_matching_question(user_question, questions):
    # Use fuzzy string matching to find the closest matching question
    matching_question = max(questions, key=lambda question: fuzz.partial_ratio(user_question, question))
    return matching_question

def search_word_information(user_input):
    # Check if the user input matches any question in QA.docx
    qa_dict = read_qa_from_docx('QA.docx')
    questions = list(qa_dict.keys())

    matching_question = find_matching_question(user_input, questions)

    if fuzz.partial_ratio(user_input, matching_question) > 60:
        print("Answer from QA System:", qa_dict[matching_question])
    else:
        # If no match is found, use GPT-3.5 Turbo to generate a response
        prompt = f"Search Information about {user_input}"

        response = client.completions.create(
            model="gpt-3.5-turbo-instruct",
            prompt=prompt,
            max_tokens=150,
            n=1,
            stop=None,
            temperature=0.7,
        )

        generated_text = response.choices[0].text
        print("Generated Information:", generated_text)

        print("I'm here to offer general information and assist you with related topics. However, I can still provide some more information. Feel free to ask another query!")

if __name__ == "__main__":
    print("Welcome to the QA system!")
    while True:
        user_input = input("Enter a word to search information about (or type 'exit' to end): ").strip().lower()

        if user_input == 'exit':
            break

        search_word_information(user_input)

Welcome to the QA system!


Enter a word to search information about (or type 'exit' to end):  exit


This Model stored the date & generate the answer

In [6]:
import os
from openai import OpenAI
from docx import Document
from fuzzywuzzy import fuzz

OpenAI.api_key = os.environ["OPENAI_API_KEY"]
client = OpenAI()

def read_qa_from_docx(docx_path):
    doc = Document(docx_path)
    qa_dict = {}

    current_question = None
    for paragraph in doc.paragraphs:
        text = paragraph.text.strip()

        if text.endswith("?"):
            current_question = text
        elif current_question and text.endswith("."):
            qa_dict[current_question] = text
            current_question = None

    return qa_dict

def find_matching_question(user_question, questions):
    matching_question = max(questions, key=lambda question: fuzz.partial_ratio(user_question, question))
    return matching_question

def add_qa_to_docx(docx_path, question, answer):
    doc = Document(docx_path)
    doc.add_paragraph(question)
    doc.add_paragraph(answer)
    doc.save(docx_path)

def search_word_information(user_input, docx_path='QA.docx'):
    qa_dict = read_qa_from_docx(docx_path)
    questions = list(qa_dict.keys())

    matching_question = find_matching_question(user_input, questions)

    if fuzz.partial_ratio(user_input, matching_question) > 60:
        print("Answer from QA System:", qa_dict[matching_question])
    else:
        prompt = f"Search Information about {user_input}"

        response = client.completions.create(
            model="gpt-3.5-turbo-instruct",
            prompt=prompt,
            max_tokens=500,
            n=1,
            stop=None,
            temperature=0.7,
        )

        generated_text = response.choices[0].text
        print("Generated Information:", generated_text)

        # Add the user's question and the generated answer to the document
        add_qa_to_docx(docx_path, user_input, generated_text)
        print('----------------------------------------------------------------------------------------------------------------------------------------------')
        print("I'm here to offer general information and assist you with related topics. However, I can still provide some more information. Feel free to ask another query!")
        print('**********************************************************************************************************************************************')
        
if __name__ == "__main__":
    print('----------------------------------------------------------------------------------------------------------------------------------------------')
    print("Welcome to the Chat Box that saved the data")
    print('----------------------------------------------------------------------------------------------------------------------------------------------')
    while True:
        user_input = input("Enter to search information about (or type 'exit' to end): ").strip().lower()
        print('----------------------------------------------------------------------------------------------------------------------------------------------')

        if user_input == 'exit':
            print('Thank You')
            break
        search_word_information(user_input)

----------------------------------------------------------------------------------------------------------------------------------------------
Welcome to the Chat Box that saved the data
----------------------------------------------------------------------------------------------------------------------------------------------


Enter to search information about (or type 'exit' to end):  exit


----------------------------------------------------------------------------------------------------------------------------------------------
Thank You


This model stored the data in Excel format , CSV format

In [7]:
import os
import pandas as pd
from openai import OpenAI
from fuzzywuzzy import fuzz

OpenAI.api_key = os.environ["OPENAI_API_KEY"]
client = OpenAI()

def read_qa_from_excel(excel_path):
    df = pd.read_excel(excel_path)
    return df

def add_qa_to_excel(excel_path, question, answer):
    df = pd.read_excel(excel_path)
    new_row = pd.DataFrame({"Questions": [question], "Answers": [answer]})
    df = pd.concat([df, new_row], ignore_index=True)
    df.to_excel(excel_path, index=False)

def find_matching_question(user_question, questions):
    if not questions:
        return None  # Return None if the list of questions is empty

    matching_question = max(questions, key=lambda question: fuzz.partial_ratio(user_question, question))
    return matching_question

def search_word_information(user_input, excel_path='QA.xlsx'):
    qa_df = read_qa_from_excel(excel_path)
    questions = list(qa_df["Questions"])

    matching_question = find_matching_question(user_input, questions)

    if matching_question is not None and fuzz.partial_ratio(user_input, matching_question) > 60:
        answer = qa_df.loc[qa_df["Questions"] == matching_question, "Answers"].values[0]
        print("Answer from QA System:", answer)
    else:
        prompt = f"Search Information about {user_input}"

        response = client.completions.create(
            model="gpt-3.5-turbo-instruct",
            prompt=prompt,
            max_tokens=500,
            n=1,
            stop=None,
            temperature=0.7,
        )

        generated_text = response.choices[0].text
        print("Generated Information:", generated_text)

        # Add the user's question and the generated answer to the Excel file
        add_qa_to_excel(excel_path, user_input, generated_text)
        print('----------------------------------------------------------------------------------------------------------------------------------------------')
        print("I'm here to offer general information and assist you with related topics. However, I can still provide some more information. Feel free to ask another query!")
        print('**********************************************************************************************************************************************')

if __name__ == "__main__":
    print('----------------------------------------------------------------------------------------------------------------------------------------------')
    print("Welcome to the Chat Bot")
    print('----------------------------------------------------------------------------------------------------------------------------------------------')
    while True:
        user_input = input("Enter to search information about (or type 'exit' to end): ").strip().lower()
        print('----------------------------------------------------------------------------------------------------------------------------------------------')

        if user_input == 'exit':
            print('Thank You')
            break
        search_word_information(user_input)

----------------------------------------------------------------------------------------------------------------------------------------------
Welcome to the Chat Bot
----------------------------------------------------------------------------------------------------------------------------------------------


Enter to search information about (or type 'exit' to end):  exit


----------------------------------------------------------------------------------------------------------------------------------------------
Thank You
